In [62]:
#this notebook will be mainly used for the capstone project

import pandas as pd

#we import the data
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [63]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
#Remove Borough where it is not assigned, Neighbourhood is already classified by postal code
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [65]:
# We look for not assigned neighbourhood => none of them is detetected
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [66]:
#We check the dataframe size
df.shape

(103, 3)

In [67]:
#We import the location data
df2=pd.read_csv("http://cocl.us/Geospatial_data")

In [68]:
#We merge the two tables with postal code
dfinal = df.merge(df2, on="Postal Code", how = 'inner')

In [69]:
dfinal.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [70]:
#We install folium
! pip install folium===0.5.0

In [71]:
import folium

In [72]:
#We create the toronto map
map_toronto = folium.Map(location=[43.74893391, -79.41817098], zoom_start=10)

In [73]:
dfinal.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [74]:
#We show first all the neighbourhood location by postal code in the map
for index, row in dfinal.iterrows():
    #print(row['Postal Code'], row['Latitude'], row['Longitude'])
    folium.CircleMarker(
    [row['Latitude'], row['Longitude']],
    radius=5,
    popup=row['Postal Code'],
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto) 

In [75]:
map_toronto

In [76]:
#We cluster the neighbourhood by location. I choose five cluster
from sklearn.cluster import KMeans
import numpy as np

kclusters = 5

X = dfinal['Latitude'].values.tolist()
Y = dfinal['Longitude'].values.tolist()

Z = []
ind = 0
for i in X:
    Z += [[i,Y[ind]]]
    ind += 1

X_final = np.array(Z)
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_final)

position_cluster = kmeans.cluster_centers_

#We display the position of the five clusters
position_cluster

array([[ 43.74893391, -79.41817098],
       [ 43.68190854, -79.52763743],
       [ 43.66287322, -79.39393077],
       [ 43.78055495, -79.22311504],
       [ 43.72297114, -79.3148805 ]])

In [77]:
#We display the five cluster in the map
for i in position_cluster:
    
    folium.CircleMarker(
    [i[0], i[1]],
    radius=10,
    popup=row['Postal Code'],
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto) 

In [78]:
map_toronto